# Working Viewer Demo - Exact Pattern from localhost:9000

This notebook replicates the **exact pattern** used by `simple_static_display.py` which generates the working viewer at http://localhost:9000.

**Working viewer**: http://localhost:9000 ← Generated by simple_static_display.py

This notebook will:
- ✅ Use the SAME display creation pattern
- ✅ Use explicit FactorMeta/NumberMeta (not infer_metas)
- ✅ Generate identical file structure
- ✅ Show how to start server manually (like the working version)

## What Makes localhost:9000 Work

The key is using the **exact same pattern** that's been debugged and verified.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import json

from trelliscope import Display
from trelliscope.meta import FactorMeta, NumberMeta

## 1. Inspect the Working Display at localhost:9000

Let's see what `simple_static_display.py` generated.

In [ ]:
# Path to the working display
working_display_path = Path("output/simple_static_test")

print("Working Display at http://localhost:9000")
print("="*60)
print(f"Generated by: simple_static_display.py")
print(f"Location: {working_display_path}")
print()

# Check structure
print("Structure:")
for file in ["index.html", "config.json"]:
    exists = "✓" if (working_display_path / file).exists() else "✗"
    print(f"  {exists} {file}")

display_dir = working_display_path / "displays" / "simple_static"
for file in ["displayInfo.json", "metaData.json", "metaData.js", "metadata.csv"]:
    exists = "✓" if (display_dir / file).exists() else "✗"
    print(f"  {exists} displays/simple_static/{file}")

panels_dir = display_dir / "panels"
panel_count = len(list(panels_dir.glob("*.png"))) if panels_dir.exists() else 0
print(f"  ✓ {panel_count} panels in displays/simple_static/panels/")

print()
print("🌐 Currently serving at: http://localhost:9000")
print("="*60)

## 2. The Exact Working Pattern

Now let's replicate the **exact pattern** from `simple_static_display.py`.

### Key Pattern Elements:
1. Create data as dict, then convert to DataFrame
2. Use explicit `FactorMeta` and `NumberMeta` (NOT `infer_metas()`)
3. Call methods individually (not method chaining)
4. Write with `output_path` parameter
5. Start server manually with `python -m http.server`

In [ ]:
# EXACT PATTERN from simple_static_display.py

def create_simple_plot(category, value):
    """Create a simple bar plot - EXACT function from working version."""
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.bar([category], [value], color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'][ord(category) - ord('A')])
    ax.set_ylim(0, 35)
    ax.set_title(f"Category {category}")
    ax.set_ylabel("Value")
    plt.tight_layout()
    return fig

# Create data - EXACT pattern: dict first, then DataFrame
data = {
    "category": ["A", "B", "C", "D", "E"],
    "value": [10, 25, 15, 30, 20],
    "panel": []  # Will be populated with figures
}

# Create matplotlib figures - EXACT loop pattern
for cat, val in zip(data["category"], data["value"]):
    fig = create_simple_plot(cat, val)
    data["panel"].append(fig)

# Convert to DataFrame
df = pd.DataFrame(data)

print("Data created:")
print(df[["category", "value"]].to_string())

In [ ]:
# Create display - EXACT pattern: no method chaining initially
print("Creating display...")
display = Display(df, name="notebook_demo", description="Notebook Demo - Exact Working Pattern")

# Set panel column
display.set_panel_column("panel")

# Add meta variables - EXPLICIT FactorMeta and NumberMeta (KEY DIFFERENCE!)
display.add_meta_variable(
    FactorMeta(varname="category", label="Category", levels=["A", "B", "C", "D", "E"])
)
display.add_meta_variable(
    NumberMeta(varname="value", label="Value")
)

# Set default layout - EXACT parameters
display.set_default_layout(ncol=3, nrow=None, arrangement="row")
display.set_default_labels(["category", "value"])

print(f"Display created: {display.name}")
print(f"Panel column: {display.panel_column}")
print(f"Meta variables: {display.list_meta_variables()}")

In [ ]:
# Write display - EXACT pattern with output_path parameter
output_dir = Path("output/notebook_demo")
print(f"Writing display to {output_dir}...")

display.write(output_path=output_dir, force=True, viewer_debug=False)

print(f"✓ Display written to: {output_dir}")

# Close matplotlib figures
plt.close('all')

## 3. Verify Generated Files

Let's verify the structure matches the working version exactly.

In [ ]:
# Verify files - EXACT verification pattern from simple_static_display.py
print("\nVerifying files...")

# Root files
root_files = [
    output_dir / "index.html",
    output_dir / "config.json",
]

# Display directory files
display_dir = output_dir / "displays" / "notebook_demo"
display_files = [
    output_dir / "displays" / "displayList.json",
    display_dir / "displayInfo.json",
    display_dir / "metaData.json",
    display_dir / "metaData.js",
    display_dir / "metadata.csv",
]

# Panel files
panel_files = [
    display_dir / "panels" / "0.png",
    display_dir / "panels" / "1.png",
    display_dir / "panels" / "2.png",
    display_dir / "panels" / "3.png",
    display_dir / "panels" / "4.png",
]

required_files = root_files + display_files + panel_files

all_exist = True
for file_path in required_files:
    exists = file_path.exists()
    status = "✓" if exists else "✗"
    try:
        rel_path = file_path.relative_to(output_dir)
    except ValueError:
        rel_path = file_path
    print(f"  {status} {rel_path}")
    if not exists:
        all_exist = False

if all_exist:
    print("\n✓ All required files created successfully!")
    print(f"\nStructure matches working version at localhost:9000 ✅")
else:
    print("\n✗ Some files are missing!")

## 4. Compare Configuration with Working Version

Let's verify our configuration matches localhost:9000 exactly.

In [ ]:
# Read both displayInfo files
working_info_path = Path("output/simple_static_test/displays/simple_static/displayInfo.json")
new_info_path = output_dir / "displays" / "notebook_demo" / "displayInfo.json"

with open(working_info_path) as f:
    working_info = json.load(f)

with open(new_info_path) as f:
    new_info = json.load(f)

print("Configuration Comparison:")
print("="*60)

# Compare key fields
comparisons = [
    ("panelInterface.type", working_info['panelInterface']['type'], new_info['panelInterface']['type']),
    ("panelInterface.base", working_info['panelInterface']['base'], new_info['panelInterface']['base']),
    ("panelInterface.panelCol", working_info['panelInterface']['panelCol'], new_info['panelInterface']['panelCol']),
    ("primarypanel", working_info['primarypanel'], new_info['primarypanel']),
    ("n (panel count)", working_info['n'], new_info['n']),
    ("width", working_info['width'], new_info['width']),
    ("height", working_info['height'], new_info['height']),
]

all_match = True
for field, working_val, new_val in comparisons:
    match = working_val == new_val
    status = "✓" if match else "✗"
    print(f"{status} {field}:")
    print(f"    Working: {working_val}")
    print(f"    New:     {new_val}")
    if not match:
        all_match = False

# Compare meta count
working_meta_count = len(working_info['metas'])
new_meta_count = len(new_info['metas'])
match = working_meta_count == new_meta_count
status = "✓" if match else "✗"
print(f"{status} Meta variable count:")
print(f"    Working: {working_meta_count} metas")
print(f"    New:     {new_meta_count} metas")

print("\n" + "="*60)
if all_match:
    print("✅ Configuration matches working version exactly!")
else:
    print("⚠️  Some differences found (may be expected)")
print("="*60)

## 5. Verify Panel Meta Configuration

The critical panel meta must be in the metas array.

In [ ]:
# Check panel meta in new display
panel_meta = None
for meta in new_info['metas']:
    if meta['varname'] == 'panel':
        panel_meta = meta
        break

print("Panel Meta Verification:")
print("="*60)
if panel_meta:
    print("✓ Panel meta found in metas array")
    print("\nPanel meta configuration:")
    print(json.dumps(panel_meta, indent=2))
    
    # Verify critical fields
    checks = [
        (panel_meta['type'] == 'panel', f"type = '{panel_meta['type']}'"),
        (panel_meta['paneltype'] == 'img', f"paneltype = '{panel_meta['paneltype']}'"),
        (panel_meta['source']['type'] == 'file', f"source.type = '{panel_meta['source']['type']}'"),
        (panel_meta['source']['isLocal'] == True, f"source.isLocal = {panel_meta['source']['isLocal']}"),
    ]
    
    print("\nCritical field checks:")
    for passed, msg in checks:
        status = "✓" if passed else "✗"
        print(f"  {status} {msg}")
else:
    print("✗ Panel meta NOT FOUND - panels will not display!")

print("="*60)

## 6. How to View Your Display

Use the **same method** as the working version - manual HTTP server.

### Method 1: Manual Server (like localhost:9000)

Open a terminal and run:
```bash
cd examples/output/notebook_demo
python -m http.server 8765
```

Then open: http://localhost:8765/

### Method 2: Use Display.view() (with fixed code)

Run the cell below to launch the viewer programmatically.

In [ ]:
# Launch viewer using Display.view() method
# This uses the FIXED code that serves from the correct root directory

url = display.view(port=8765, open_browser=True)

print(f"\n{'='*60}")
print(f"Viewer launched at: {url}")
print(f"{'='*60}")
print("\nExpected Result:")
print("  • Should show 5 panels (same as localhost:9000)")
print("  • Panels should display correctly (not '0 of 0')")
print("  • Can filter by category (A, B, C, D, E)")
print("  • Can sort by value")
print("\nServer running in background.")
print("To stop: Restart kernel or close terminal.")

## 7. Compare All Working Viewers

You now have multiple viewers to compare.

In [ ]:
print("Working Viewer URLs:")
print("="*60)
print()
print("1. Original Working Example (simple_static):")
print("   🌐 http://localhost:9000")
print("   • Generated by: simple_static_display.py")
print("   • Method: Manual http.server")
print("   • Panels: 5 bar charts")
print("   • Status: ✅ WORKING")
print()
print("2. Notebook Demo (notebook_demo):")
print("   🌐 http://localhost:8765")
print("   • Generated by: This notebook (cell above)")
print("   • Method: Display.view() with fixed code")
print("   • Panels: 5 bar charts (same as original)")
print("   • Status: ✅ Should match original exactly")
print()
print("3. Reference Implementation:")
print("   🌐 http://localhost:8001")
print("   • Status: ✅ WORKING")
print()
print("="*60)
print("All should show panels correctly!")
print("="*60)

## 8. Key Differences: Working Pattern vs. Common Mistakes

### ✅ WORKING PATTERN (This Notebook)

```python
# 1. Create data as dict, then DataFrame
data = {"category": [...], "value": [...], "panel": []}
for cat, val in zip(data["category"], data["value"]):
    data["panel"].append(create_plot(cat, val))
df = pd.DataFrame(data)

# 2. Create display (no path in constructor for simplicity)
display = Display(df, name="my_display", description="...")

# 3. Set panel column
display.set_panel_column("panel")

# 4. EXPLICIT meta variables (KEY!)
display.add_meta_variable(FactorMeta(varname="category", levels=[...]))
display.add_meta_variable(NumberMeta(varname="value"))

# 5. Set layout
display.set_default_layout(ncol=3, nrow=None, arrangement="row")
display.set_default_labels(["category", "value"])

# 6. Write with output_path
display.write(output_path=Path("output/my_display"), force=True)

# 7. View (optional)
display.view(port=8765)
```

### ❌ COMMON MISTAKES

```python
# ❌ Using infer_metas() instead of explicit metas
display.infer_metas()  # May work, but not the proven pattern

# ❌ Not setting panel column
# display.set_panel_column("panel")  # MUST call this!

# ❌ Wrong write() usage
display.write()  # Missing output_path

# ❌ Old panelInterface (in old code)
# {"type": "panel_local", "base": "./panels"}  # Wrong!
```

### Why Explicit Metas?

- `.infer_metas()` SHOULD work (and does in most cases)
- But the PROVEN working pattern uses explicit `FactorMeta`/`NumberMeta`
- Explicit metas give you more control over labels, levels, formatting
- Matches the R trelliscope pattern exactly
- No ambiguity about meta types

## 9. Troubleshooting Checklist

If you see "0 of 0 panels" or blank viewer:

1. ✅ **Panel column set?**
   - Must call `.set_panel_column("panel")`

2. ✅ **Meta variables added?**
   - Use explicit `FactorMeta`/`NumberMeta` OR `.infer_metas()`

3. ✅ **Files generated?**
   - Check `output_path/config.json` exists
   - Check `output_path/displays/displayList.json` exists
   - Check `output_path/displays/{name}/displayInfo.json` exists

4. ✅ **Panel files exist?**
   - Check `output_path/displays/{name}/panels/*.png` exist

5. ✅ **panelInterface correct?**
   - type == "file" (not "panel_local")
   - base == "panels" (not "./panels")

6. ✅ **Server from correct directory?**
   - Should serve from `output_path/` (root)
   - NOT from `output_path/displays/`

7. ✅ **Browser console errors?**
   - Open browser DevTools (F12)
   - Check Console tab for errors
   - Check Network tab for 404s

## Summary

This notebook demonstrated:

1. ✅ **Exact working pattern** from `simple_static_display.py`
2. ✅ **Explicit meta variables** (FactorMeta/NumberMeta)
3. ✅ **Correct file structure** matching localhost:9000
4. ✅ **Verified configuration** matches working version
5. ✅ **Launched viewer** with fixed Display.view() code

Your notebook display at **http://localhost:8765** should now match the working version at **http://localhost:9000** exactly! 🎉

### Key Takeaway

Use the **proven pattern** from `simple_static_display.py`:
- Explicit `FactorMeta`/`NumberMeta` 
- Individual method calls (not always method chaining)
- `output_path` parameter in `.write()`
- Manual server OR `.view()` with fixed code